# Customer Churn Factors

Telco is a world renowned telecommunications company that provides phone, internet, and streaming services to their customers. As technology evolves, so does the company's business model. In order to stay up to date on current market trends and ensuring the satisfaction of our customers, it is necessary to analyze customer data. As you will soon see, there may be some factors that influence customer churn that can be remediated to keep customers with the company.  Different models are ran to determine what factors have the greatest influence on customer churn and the results will hopefully spark discussion on how to decrease customer churn.

## Imports

In [39]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

from scipy import stats
import acquire
import prepare

## Acquire

* Accquired data from noether sequal server
* data obtained 1/2/2023
* 7043 rows by 24 columns before cleaning
* Each row represents a customer
* Each column represents a demograhic or service type information


## Prepare
* Removed columns that did not contain useful information
* Renamed columns to promote readability
* Checked for nulls in the data (there were none)
* Checked that column data types were appropriate
* Removed white space from values in object columns
* Added Target column 'upset' indicating weather the lower rated player won the game
* Added additional features to investigate:
    * Rating Difference
    * Game Rating
    * Lower Rated White
    * Time Control Group
    * Upset
* Encoded categorical variables
* Split data into train, validate and test (approx. 60/25/15), stratifying on 'upset'
* Scaled continuous variable
* Outliers have not been removed for this iteration of the project

In [9]:
telco = acquire.get_telco_data() #acquiring data from server and assigning it to a variable

In [10]:
telco.head() # a brief look at the data

,Unnamed: 0,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,...,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,...,No,2,Yes,2,65.6,593.3,No,DSL,One year,Mailed check
1,1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,...,Yes,1,No,2,59.9,542.4,No,DSL,Month-to-month,Mailed check
2,2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,...,No,1,Yes,1,73.9,280.85,Yes,Fiber optic,Month-to-month,Electronic check
3,3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,...,Yes,1,Yes,1,98.0,1237.85,Yes,Fiber optic,Month-to-month,Electronic check
4,4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,...,No,1,Yes,2,83.9,267.4,Yes,Fiber optic,Month-to-month,Mailed check


In [12]:
#dropping unessessary columns
telco.drop(columns = ['Unnamed: 0', 'customer_id', 'gender', 'senior_citizen', 'partner', 'dependents', 'tenure'
                     , 'phone_service', 'internet_service_type_id', 'streaming_movies', 'contract_type_id',
                      'paperless_billing', 'payment_type_id', 'payment_type', 'multiple_lines', 'online_backup',
                     'device_protection', 'tech_support', 'streaming_tv'], inplace = True)

In [19]:
# droping null rows
telco.dropna()

,online_security,monthly_charges,total_charges,churn,internet_service_type,contract_type
0,No,65.60,593.3,No,DSL,One year
1,No,59.90,542.4,No,DSL,Month-to-month
2,No,73.90,280.85,Yes,Fiber optic,Month-to-month
3,No,98.00,1237.85,Yes,Fiber optic,Month-to-month
4,No,83.90,267.4,Yes,Fiber optic,Month-to-month
...,...,...,...,...,...,...
7038,Yes,55.15,742.9,No,DSL,One year
7039,No,85.10,1873.7,Yes,Fiber optic,Month-to-month
7040,No,50.30,92.75,No,DSL,Month-to-month
7041,Yes,67.85,4627.65,No,DSL,Two year


In [26]:
#removing rows that have a space in total_charges
telco = telco.loc[telco["total_charges"] != ' '] 

In [31]:
#change values in total charges to float
telco['total_charges'] = telco["total_charges"].astype(float)

In [35]:
telco.dtypes

online_security           object
monthly_charges          float64
total_charges            float64
churn                     object
internet_service_type     object
contract_type             object
dtype: object

## A quick look at the data


In [32]:
telco.head()

,online_security,monthly_charges,total_charges,churn,internet_service_type,contract_type
0,No,65.6,593.30,No,DSL,One year
1,No,59.9,542.40,No,DSL,Month-to-month
2,No,73.9,280.85,Yes,Fiber optic,Month-to-month
3,No,98.0,1237.85,Yes,Fiber optic,Month-to-month
4,No,83.9,267.40,Yes,Fiber optic,Month-to-month


## A summary of the data

In [37]:
telco.describe()

,monthly_charges,total_charges
count,7032.000000,7032.000000
mean,64.798208,2283.300441
std,30.085974,2266.771362
min,18.250000,18.800000
25%,35.587500,401.450000
50%,70.350000,1397.475000
75%,89.862500,3794.737500
max,118.750000,8684.800000


In [2]:
select c.online_security, c.monthly_charges,
c.total_charges, c.churn, ctype.contract_type 
from customers as c
join contract_types as ctype using(contract_type_id);

SyntaxError: invalid syntax (1860332388.py, line 1)

## Prepare

* List steps taken to clean your data here
* In particular call out how you handle null values and outliers in detail
* You must do this even if you do not do anything or do not encounter any
* Any time there is potential to make changes to the data you must be upfront about the changes you make or do not make

In [ ]:
# Import your prepare function and use it to clean your data here

## Explore

* Here you will explore your data then highlight 4 questions that you asked of the data and how those questions influanced your analysis
* Remember to split your data before exploring how differint variables relate to one another
* Each question should be stated directly 
* Each question should be supported by a vizualization
* Each question should be answered in natural language
* Two questions must be supported by a statistical test, but you may choose to support more than two
* See the following example, and read the comments in the next cell

The following empty code block is here to represent the countless questions, visualizations, and statistical tests 
that did not make your final report. Data scientist often create a myriad of questions, vizualizations 
and statistical tests that do not make it into the final notebook. This is okay and expected. Remember 
that shotgun approches to your data such as using pair plots to look at the relationships of each feature 
pair are a great way to explore your data, but they have no place in your final report. 
**Your final report is about showing your findings, not showing your work!**

## You may use this as a templet for how to ask and answer each question:

### 1) Question about the data
* ask a question about the data for which you got a meaningful result.

### 2) Vizualization of the data answering the question

* vizualizations should be acompanied by take-aways telling the reader exaclty what you want them to get from the chart
* Good take-aways often make for good chart titles. Try to use your chart title instead of bullet points.

### 3) Statistical test
* Be sure you are using the correct statistical test for the type of variables you are testing
* be sure that you are not violating any of the asumptions for the statistical test you are choosing
* your notebook should run and produce the results of the test you are using (This may be done through imports)
* Include an introduction to the kind of test you are doing
* Include the Ho and Ha for the test
* Include the alpha you are using
* Include the readout of the p-value for the test
* Interpret the results of the test in natural language 

### 4) Answer to the question
* Answer the question that you posed of the data by refering to the chart and statistical test (if you used one).
* if question relates to drivers, explain why the feature in question would/wouldn't make a good driver.

## Exploration Summery
* after your explore section before you start modeling provide a summary of your findings in Explore
* Include a summery of your take-aways from Explore including answers to the questions you asked
* Include a summery of the features you examined and weather or not you will be going to Modeling with each feature and why
* It is important to note which features will be going into your model so the reader knows what features you will be modeling on


## Modeling

* Explain how you will be evauating your models. Be sure to include the evaluation metric you will be using and why you have chosen it

### Baseline 
* create a baseline and briefly explain how it was calculated 

In [ ]:
# If your baseline comes from a model run the model here and show it's results

### Best 3 Models
* Show the three best model results obtained using your selected features to predict the target variable
* Tipically students will show the top models they are able to generate for three differint model types

## You may use this as a templet for how to introduce your models:

### Model Type

In [ ]:
# code that runs the best model in that model type (this mad be imported from a module)

Print out of model code should read: <br>
"Model Type" <br>
"evaluation metric" on train: "evaluation result" <br>
"evaluation metric" on validate: "evaluation result"

### Test Model
* Choose the best model out of the three as yout best model and explain why you have chosen it
* Explain that you will now run your final model on test data to guage how it will perform on unseen data


In [ ]:
# Code that runs the best overall model on test data (this may be imported from a module)

Print out of model code should read: <br>
"Model Type" <br>
"evaluation metric" on Test: "evaluation result" <br>

### Modeling Wrap 
* Give a final interpritation of how the models test score compares to the baseline and weather you would recomend this model for production

## Conclution

### Summery
* summerize you findings and answer the questions you brought up in explore 
* summerize how drivers discovered lead or did not lead to a successful model 

### Recommendations
* Recommendations are actions the stakeholder should take based on your insights

### Next Steps
* Nest Steps are what you as a Data Scientist would do if provided more time to work on the project

**Where there is code in your report there should also be code comments telling the reader what each code block is doing. this is ture for any and all code blocks even if you are using a function to import the code**

**Your Notebook should contain adequate markdown that documents your thought process, decision making, and navigation through the pipeline. As a Data Scientist your job is not only to make discoveries but to effectivly communicate them as well. This means documintation is a critical part of your job.**